In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.express as px
from get_data import read_data, get_data

In [2]:
import git
import sys

root_dir = git.Repo('.', search_parent_directories=True).working_tree_dir
sys.path.insert(1, root_dir)

In [3]:
df_bairros = pd.read_csv(f'{root_dir}/dados/RJ/cidade_rj/dados_covid/Dados_indiv_MRJ_covid19.csv', delimiter = ';')
df_bairros = read_data(df_bairros, 'prefeitura')
df_bairros = df_bairros.drop(columns = ['Bairro', 'AP Residência'])
df_bairros.rename(mapper = {'Faixa Etaria': 'AP Residência', 'Situação atual':'Bairro', 
                                         'Data_atualização':'Situação atual'},axis = 1, inplace = True)
df_bairros['AP Residência'].replace({'#N/D':'3.1'}, inplace = True)
df_bairros['Bairro'].replace({'COMPLEXO DA MARE': 'MARE'}, inplace = True)
df_bairros = df_bairros[df_bairros['Bairro'] != 'INDEFINIDO']
df_bairros = df_bairros[df_bairros['Bairro'] != 'FORA DO MUNICÍPIO']

In [4]:
df_bairros['Bairro'][0], df_bairros['AP Residência'][0]

('BARRA DA TIJUCA', '4.0')

In [5]:
#for i in range(len(df_bairros['Bairro'])):
#    print(list(df_bairros['AP Residência'])[i].split('.')[0])

In [6]:
bairros = df_bairros['Bairro'].unique()

In [7]:
aps_bairros = {}
for i in range(1,6):
    ap = []
    for j, b in enumerate(df_bairros['Bairro']):
        if b in ap:
            continue
        else:
            if list(df_bairros['AP Residência'])[j].split('.')[0] == str(i):
                ap.append(b)
    aps_bairros.update({i:ap})

In [8]:
# dict bairro: ap (não usamos)
'''bairros = {}
for i, b in enumerate(df_bairros['Bairro']):
    if b in bairros:
        continue
    else:
        bairros.update({b:list(df_bairros['AP Residência'])[i].split('.')[0]})'''

"bairros = {}\nfor i, b in enumerate(df_bairros['Bairro']):\n    if b in bairros:\n        continue\n    else:\n        bairros.update({b:list(df_bairros['AP Residência'])[i].split('.')[0]})"

In [9]:
#dados_bairros_acum = [get_data(b, df_bairros, 'prefeitura', '05-05-2020', T_start = '15-03-2020', to_print = False)[0] for b in bairros]
#dados_bairros_novos = [get_data(b, df_bairros, 'prefeitura', '05-05-2020', T_start = '15-03-2020', to_print = False)[1] for b in bairros]

In [10]:
dados_acum = [[get_data(b, df_bairros, 'prefeitura', '03-05-2020', 
                        T_start = '15-03-2020', to_print = False)[0] for b in aps_bairros[i]] for i in range(1,6)]
dados_novos = [[get_data(b, df_bairros, 'prefeitura', '03-05-2020', 
                        T_start = '15-03-2020', to_print = False)[1] for b in aps_bairros[i]] for i in range(1,6)]

In [11]:
for i in range(1,6):
    print(len(aps_bairros[i]))

15
25
80
18
21


In [12]:
updatemenus = list([
    dict(active=5,
         buttons=list([  
             dict(label="AP1",   
                 method="update",
                 args=[{"visible": 15*[True] + 25*[False] + 80*[False] + 18*[False] + 21*[False]},
                        {"title": "Casos <b> acumulados </b>por bairro: AP1",
                        "annotations": []}]), 
            
             dict(label = 'AP2',
                 method = 'update',
                 args = [{'visible': 15*[False] + 25*[True] + 80*[False] + 18*[False] + 21*[False]},
                         {'title': 'Casos <b> acumulados </b>por bairro: AP2',
                          'annotations': []}]),
             dict(label = 'AP3',
                 method = 'update',
                 args = [{'visible': 15*[False] + 25*[False] + 80*[True] + 18*[False] + 21*[False]},
                         {'title': 'Casos <b> acumulados </b>por bairro: AP3',
                          'annotations': []}]),
             dict(label = 'AP4',
                 method = 'update',
                 args = [{'visible': 15*[False] + 25*[False] + 80*[False] + 18*[True] + 21*[False]},
                         {'title': 'Casos <b> acumulados </b>por bairro: AP4',
                          'annotations': []}]),
             dict(label = 'AP5',
                 method = 'update',
                 args = [{'visible': 15*[False] + 25*[False] + 80*[False] + 18*[False] + 21*[True]},
                         {'title': 'Casos <b> acumulados </b>por bairro: AP5',
                          'annotations': []}]),
             dict(label = 'Todas as APs',
                 method = 'update',
                 args = [{'visible': 159*[True]},
                         {'title': 'Casos <b> acumulados </b>por bairro: Todas as APs',
                          'annotations': []}])
             
        ])
    )
])

In [13]:
symbols = ['circle', 'square', 'diamond', 'cross', 'star']
tickvals = [0,6,13,20,27,34,41,48]
ticktext = ['15/03', '22/03', '29/03', '05/04', '12/04', '19/04', '26/04', '02/05']

In [53]:
data = []
for j in range(1,6):
    for i, d in enumerate(dados_acum[j-1]):

        data.append(go.Scatter(y = d, name = aps_bairros[j][i], marker = dict(symbol = symbols[j-1]), 
                               legendgroup = aps_bairros[j][i],
                               mode = 'lines+markers'))


In [54]:
fig = go.Figure(data = data)
fig.update_layout(title = dict(text = 'Casos <b> acumulados </b> por bairro: Todas as APs', x = 0.5), 
                  updatemenus = updatemenus, xaxis = dict(tickvals = tickvals, ticktext = ticktext))
fig.show()

In [55]:
#with open("series_temporais_bairros_acum.html", "w") as f:
#    f.write(fig.to_html())

In [56]:
updatemenus2 = list([
    dict(active=5,
         buttons=list([  
             dict(label="AP1",   
                 method="update",
                 args=[{"visible": 15*[True] + 25*[False] + 80*[False] + 18*[False] + 21*[False]},
                        {"title": "Casos <b> novos </b>por bairro: AP1",
                        "annotations": []}]), 
            
             dict(label = 'AP2',
                 method = 'update',
                 args = [{'visible': 15*[False] + 25*[True] + 80*[False] + 18*[False] + 21*[False]},
                         {'title': 'Casos <b> novos </b>por bairro: AP2',
                          'annotations': []}]),
             dict(label = 'AP3',
                 method = 'update',
                 args = [{'visible': 15*[False] + 25*[False] + 80*[True] + 18*[False] + 21*[False]},
                         {'title': 'Casos <b> novos </b>por bairro: AP3',
                          'annotations': []}]),
             dict(label = 'AP4',
                 method = 'update',
                 args = [{'visible': 15*[False] + 25*[False] + 80*[False] + 18*[True] + 21*[False]},
                         {'title': 'Casos <b> novos </b>por bairro: AP4',
                          'annotations': []}]),
             dict(label = 'AP5',
                 method = 'update',
                 args = [{'visible': 15*[False] + 25*[False] + 80*[False] + 18*[False] + 21*[True]},
                         {'title': 'Casos <b> novos </b>por bairro: AP5',
                          'annotations': []}]),
             dict(label = 'Todas as APs',
                 method = 'update',
                 args = [{'visible': 159*[True]},
                         {'title': 'Casos <b> novos </b>por bairro: Todas as APs',
                          'annotations': []}])
             
        ])
    )
])

In [57]:
data2 = []
for j in range(1,6):
    for i, d in enumerate(dados_novos[j-1]):

        data2.append(go.Scatter(y = d, name = aps_bairros[j][i], marker = dict(symbol = symbols[j-1]), 
                                legendgroup = aps_bairros[j][i],
                                mode = 'lines+markers'))

In [58]:
fig2 = go.Figure(data = data2)
fig2.update_layout(title = dict(text = 'Casos <b> novos </b> por bairro: Todas as APs', x = 0.3), 
                   updatemenus = updatemenus2, xaxis = dict(tickvals = tickvals, ticktext = ticktext))
fig2.show()

In [59]:
#with open("series_temporais_bairros_novos.html", "w") as f:
#    f.write(fig.to_html())

In [60]:
df_obitos = df_bairros[df_bairros['Situação atual'] == 'óbito']

In [61]:
obitos_acum = [[get_data(b, df_obitos, 'prefeitura', '03-05-2020', 
                        T_start = '15-03-2020', to_print = False)[0] for b in aps_bairros[i]] for i in range(1,6)]
obitos_novos = [[get_data(b, df_obitos, 'prefeitura', '03-05-2020', 
                        T_start = '15-03-2020', to_print = False)[1] for b in aps_bairros[i]] for i in range(1,6)]

# mais de um botão

In [62]:
data_obito_novo = []
for j in range(1,6):
    for i, d in enumerate(obitos_novos[j-1]):

        data_obito_novo.append(go.Scatter(y = d, name = aps_bairros[j][i], 
                                          marker = dict(symbol = symbols[j-1]), legendgroup = aps_bairros[j][i], 
                                          mode = 'lines+markers'))

In [65]:
data_obito_acum = []
for j in range(1,6):
    for i, d in enumerate(obitos_acum[j-1]):

        data_obito_acum.append(go.Scatter(y = d, name = aps_bairros[j][i], 
                                          marker = dict(symbol = symbols[j-1]), legendgroup = aps_bairros[j][i], 
                                          mode = 'lines+markers'))

In [66]:
data_sum = data + data2 + data_obito_acum + data_obito_novo

In [67]:
use_confirmados = widgets.Checkbox(
    description='Confirmados: ',
    value=True,
)
use_obitos = widgets.Checkbox(
    description='Óbitos: ',
    value=False,
)
use_acumulados = widgets.Checkbox(
    description='Acumulados: ',
    value=True,
)
use_novos = widgets.Checkbox(
    description='Novos: ',
    value=False,
)

container = widgets.HBox(children=[use_confirmados, use_obitos, use_acumulados, use_novos])

textbox = widgets.Dropdown(
    description='APs:   ',
    value='AP 1',
    options=['AP 1','AP 2','AP 3','AP 4','AP 5', 'Todas as APs']
)


# Assign an empty figure widget with two traces
g = go.FigureWidget(data= data_sum,
                    layout=go.Layout(
                        title=dict(
                            text='Evolução <b> confirmados  acumulados</b> por bairro no município do Rio de Janeiro'
                        ),
                        xaxis = dict(tickvals = tickvals, ticktext = ticktext)
                    ))

In [68]:
possiveis_titulos = ['confirmados acumulados', 'confirmados novos', 
                    'óbitos acumulados', 'óbitos novos', 'confirmados e óbitos acumulados',
                    'confirmados e óbitos novos', 'confirmados', 'óbitos']

In [69]:
15
25
80
18
21

21

In [70]:
def response(change):
    booleanos = [use_confirmados.value, use_obitos.value, use_acumulados.value, use_novos.value ]
    valor_AP = textbox.value
    ap_visible = []
    if valor_AP == 'AP 1':
        for i,d in enumerate(g.data):
            ap_visible.append(i%159<15)
    
    elif valor_AP == 'AP 2':
        for i,d in enumerate(g.data):
            ap_visible.append(15 <= i%159 < 40) 
            
    elif valor_AP == 'AP 3':
        for i,d in enumerate(g.data):
            ap_visible.append(40<= i%159 <120) 
            
    elif valor_AP == 'AP 4':
        for i,d in enumerate(g.data):
            ap_visible.append(120<= i%159 <138)        
    
    elif valor_AP == 'AP 5':
        for i,d in enumerate(g.data):
            ap_visible.append(138<= i%159) 
            
    elif valor_AP == 'Todas as APs':
        ap_visible = 159*4*[True]
    
    visible_blocos = [booleanos[0] and booleanos[2], 
                      booleanos[0] and booleanos[3], 
                      booleanos[1] and booleanos[2],
                      booleanos[1] and booleanos[3]]
    lista_visible = 159*[visible_blocos[0]] + 159*[visible_blocos[1]] + 159*[visible_blocos[2]] + 159*[visible_blocos[3]]
    novos_visible = [i and j for i,j in zip(ap_visible, lista_visible)]
            

    with g.batch_update():
        #print(len(novos_visible))
        for i,d in enumerate(g.data):
            #print(i, len(g.data))
            d.visible = novos_visible[i]
        g.layout.xaxis.title = 'Dias'
        g.layout.yaxis.title = 'Ocorrências'



textbox.observe(response, names="value")
use_confirmados.observe(response, names="value")
use_obitos.observe(response, names="value")
use_acumulados.observe(response, names="value")
use_novos.observe(response, names="value")

In [72]:
container2 = widgets.HBox([textbox])
widgets.VBox([container,
              container2,
              g])

In [75]:
with open(saveWidget("teste.html"), "w") as f:
    f.write(g.to_html())

NameError: name 'saveWidget' is not defined